In [ ]:
import numpy as np
from PIL import Image
from PIL.TiffTags import TAGS
# print(TAGS)

In [ ]:
src = "/home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_imgs"
fnms = [("kit", "kit/FeMoOx_AntiA_04_1k5x_CN.tif"),
        ("ikz_changming", "ikz_changming/STO252b Camera Ceta 1818 790 kx.png"),
        ("axon", "axon/20210426T224437.049Raw0.png"),
        ("ikz_robert_t1", "ikz_robert/0c8nA_3deg_003_AplusB_test.tif"),
        ("ikz_martin_t2", "ikz_robert/ALN_baoh_021.tif"),
        ("ikz_robert_t3", "ikz_robert/T3_image.tif"),
        ("ikz_robert_etd", "ikz_robert/ETD_image.tif"),
        ("ikz_robert_navcam", "ikz_robert/NavCam_normal_vis_light_ccd.tif")
       ]
fnm = f"{src}/{fnms[2][1]}"
print(fnm)

***

In [ ]:
all_parent_concepts = set()
for idx in np.arange(3, len(fnms)):
    fnm = f"{src}/{fnms[idx][1]}"
    # print(f"{fnm}")
    with Image.open(fnm, mode="r") as fp:
        if True is False:
            czi_keys = [34118, 34119]
            for czi_key in czi_keys:
                if czi_key in fp.tag_v2:
                    print(f"Found czi_key {tfs_key}...")
                    utf = fp.tag[czi_key]
                    print(type(utf))
                    if len(utf) == 1:
                        print(utf[0])
        # exit(1)
        tfs_keys = [34682]
        for tfs_key in tfs_keys:
            if tfs_key in fp.tag_v2:
                # print(f"Found tfs_key {tfs_key}...")
                utf = fp.tag[tfs_key]
                # print(type(utf))
                if len(utf) == 1:
                    print(utf[0])
                    for line in utf[0].splitlines():
                        if line.startswith("[") is True:
                            all_parent_concepts.add(line)

for concept in all_parent_concepts:
    print(concept)

In [ ]:
# fish all concepts

In [ ]:
def sort_ascendingly_by_second_argument(tup):
    # convert the list of tuples to a numpy array with data type (object, int)
    arr = np.array(tup, dtype=[('col1', object), ('col2', int)])
    # get the indices that would sort the array based on the second column
    indices = np.argsort(arr['col2'])
    # use the resulting indices to sort the array
    sorted_arr = arr[indices]
    # convert the sorted numpy array back to a list of tuples
    sorted_tup = [(row['col1'], row['col2']) for row in sorted_arr]
    return sorted_tup

In [ ]:
import mmap
TiffTfsParentConcepts = ["Accessories", 
                         "EasyLift",
                         "ETD",
                         "Specimen",
                         "T3",
                         "User",
                         "Scan",
                         "T1",
                         "Beam",
                         "Image",
                         "PrivateFei",
                         "EBeamDeceleration",
                         "EBeam",
                         "T2",
                         "CompoundLensFilter",
                         "Stage",
                         "Nav-Cam",
                         "EScan",
                         "HotStageMEMS",
                         "ColdStage",
                         "HotStage",
                         "System",
                         "GIS",
                         "Vacuum",
                         "IRBeam",
                         "HotStageHVHS",
                         "HiResIllumination",
                         "Detectors"]

all_concepts = set()
for idx in np.arange(3, len(fnms)):
    fnm = f"{src}/{fnms[idx][1]}"
    # print(f"{fnm}")
    with open(fnm, 'rb', 0) as fp:
        s = mmap.mmap(fp.fileno(), 0, access=mmap.ACCESS_READ)
        s.seek(0, 2)
        fsize = s.tell()
        print(fsize)
        s.seek(0, 0)
        tfs_parent_concepts_byte_offset = {}
        for concept in TiffTfsParentConcepts:
            pos = s.find(bytes(f"[{concept}]", "utf8"))  # != -1
            if pos != -1:
                tfs_parent_concepts_byte_offset[concept] = pos
            # else:
                # print(f"Instance of concept [{concept}] was not found !")
        # print(tfs_parent_concepts_byte_offset)

        sequence = []  # decide I/O order in which metadata for childs of parent concepts will be read
        for key, value in tfs_parent_concepts_byte_offset.items():
            if value is not None:
                sequence.append((key, value))
                # tuple of parent_concept name and byte offset
        sequence = sort_ascendingly_by_second_argument(sequence)
        # print(sequence)

        idx = 0
        for parent, byte_offset in sequence:
            pos_s = byte_offset
            pos_e = None
            if idx < len(sequence) - 1:
                pos_e = sequence[idx + 1][1]
            else:
                pos_e = fsize  # np.iinfo(np.uint64).max
                # TODO::better use official convention to not read beyond the end of file
            idx += 1
            if pos_s is None or pos_e is None:
                raise ValueError(f"Definition of byte boundaries for reading childs of [{parent}] was unsuccessful !")
            print(f"Search for [{parent}] in between byte offsets {pos_s} and {pos_e}")
            s.seek(pos_s, 0)
            payload = s.read(pos_e - pos_s).decode("utf8")
            for line in payload.split("\n"):
                # print(line)
                if line.count("=") == 1:
                    all_concepts.add(f"{parent}/{line.split('=')[0]}")
            # print(payload)

for concept in all_concepts:
    print(concept)

In [ ]:
# handle TIFF
with Image.open(fnm, mode="r") as fp:
    for key in fp.tag_v2:
        if key in [34118, 34119]:
            print(type(fp.tag[key]))
            print(len(fp.tag[key]))        
            # print(f"{key}, {fp.tag[key]}")
        if key not in TAGS.keys():
            print(f"--->tag {key}, is not in PIL.TiffTAGS !")
    # self.tags = {TAGS[key] : fp.tag[key] for key in fp.tag_v2}
    # for key, val in self.tags.items():
    #     print(f"{key}, {val}")
    nparr = np.array(fp)
    print(f"{type(nparr)}")
    print(f"{nparr.dtype}")
    print(f"{np.shape(nparr)}")

In [ ]:
import numpy as np
# https://www.geeksforgeeks.org/python-program-to-sort-a-list-of-tuples-by-second-item/
def sort_tuple(tup):
    # convert the list of tuples to a numpy array with data type (object, int)
    arr = np.array(tup, dtype=[('col1', object), ('col2', int)])
    # get the indices that would sort the array based on the second column
    indices = np.argsort(arr['col2'])
    # use the resulting indices to sort the array
    sorted_arr = arr[indices]
    # convert the sorted numpy array back to a list of tuples
    sorted_tup = [(row['col1'], row['col2']) for row in sorted_arr]
    return sorted_tup

In [ ]:
def if_str_represents_float(s):
    try:
        return isinstance(float(s), float)
        # return str(float(s)) == s
    except ValueError:
        return False

In [ ]:
"10".isdigit()
# isinstance(float("8.99306e-010"), float)


In [ ]:
# TFS sections based on IKZ ALN_baoh_021.tif example
import mmap

tfs_section_names = ["[User]",
                     "[System]",
                     "[Beam]",
                     "[EBeam]",                 
                     "[GIS]",
                     "[Scan]",
                     "[EScan]",
                     "[Stage]",
                     "[Image]",
                     "[Vacuum]",
                     "[Specimen]",
                     "[Detectors]",
                     "[T2]",
                     "[Accessories]",
                     "[EBeamDeceleration]",
                     "[CompoundLensFilter]",
                     "[PrivateFei]",
                     "[HiResIllumination]",
                     "[EasyLift]",
                     "[HotStageMEMS]",
                     "[HotStage]",
                     "[HotStageHVHS]",
                     "[ColdStage]"]

tfs_section_details = {"[System]": ["Type", "Dnumber", "Software", "BuildNr", "Source", "Column", "FinalLens", "Chamber", "Stage", "Pump",
              "ESEM", "Aperture", "Scan", "Acq", "EucWD", "SystemType", "DisplayWidth", "DisplayHeight"]}
tfs_section_offsets = {}

with open(fnm, 'rb', 0) as file:
    s = mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ)
    for section_name in tfs_section_names:
        pos = s.find(bytes(section_name, "utf8"))  # != -1
        tfs_section_offsets[section_name] = pos
    print(tfs_section_offsets)

    # define search offsets
    tpl = []
    for key, value in tfs_section_offsets.items():
        tpl.append((key, value))
    # print(tpl)
    tpl = sort_tuple(tpl)
    print(tpl)
    # if section_name == "[System]":
    pos_s = None
    pos_e = None
    for idx in np.arange(0, len(tpl)):
        if tpl[idx][0] != "[System]":
            continue
        else:
            pos_s = tpl[idx][1]
            if idx <= len(tpl) - 1:
                pos_e = tpl[idx + 1][1]
            break
    print(f"Search in between byte offsets {pos_s} and {pos_e}")
    # fish metadata of e.g. the system section
    section_metadata = {}
    for term in tfs_section_details["[System]"]:
        
        s.seek(pos_s, 0)
        pos = s.find(bytes(term, "utf8"))
        if pos < pos_e:  # check if pos_e is None
            s.seek(pos, 0)
            section_metadata[f"{term}"] = f"{s.readline().strip().decode('utf8').replace(f'{term}=', '')}"
            if if_str_represents_float(section_metadata[f"{term}"]) is True:
                section_metadata[f"{term}"] = np.float64(section_metadata[f"{term}"])
            elif section_metadata[f"{term}"].isdigit() is True:
                section_metadata[f"{term}"] = np.int64(section_metadata[f"{term}"])
            else:
                pass
            # print(f"{term}, {pos}, {pos + len(term) + 1}")
    #        tfs_section_offswr
    #        file.seek(pos, 0)  #
    print(section_metadata)

In [ ]:
'2'.isdigit()

In [ ]:
# https://codereview.stackexchange.com/a/21035
# https://stackoverflow.com/questions/38852822/how-to-flatten-xml-file-in-python
import xmltodict
from collections import OrderedDict

def flatten_dict(d):
    def items():
        for key, value in d.items():
            # nested subtree
            if isinstance(value, dict):
                for subkey, subvalue in flatten_dict(value).items():
                    yield '{}.{}'.format(key, subkey), subvalue
            # nested list
            elif isinstance(value, list):
                for num, elem in enumerate(value):
                    for subkey, subvalue in flatten_dict(elem).items():
                        yield '{}.[{}].{}'.format(key, num, subkey), subvalue
            # everything else (only leafs should remain)
            else:
                yield key, value
    return OrderedDict(items())

In [ ]:
import re
def get_protochip_variadic_concept(token):
    # return val is a tuple
    # zeroth entry specifies the prefix of the instance concept
    # first entry specifies associated parent concept
    # second entry is True is token resolves an AXON parent concept
    if token.count(" = ") == 1:
        tmp = token.split("=")
        concept = tmp[0].strip()
        value = tmp[1].strip()
        # print(f"{concept}, {value}")
        idxs = re.finditer(".\[[0-9]+\].", concept)
        is_variadic = False
        if (sum(1 for _ in idxs) > 0):  #  and ("DataValues" in concept):
            is_variadic = True
            variadic = concept
            concept_id = []
            for idx in re.finditer(".\[[0-9]+\].", token):
                variadic = variadic.replace(concept[idx.start(0):idx.end(0)], ".[*].")
                concept_id.append(f"{concept[idx.start(0):idx.end(0)]}".replace(".[", "").replace("].", ""))
            if ("DataValues" in concept) and concept.count(".Name = ") == 1:
                return (variadic, f"{variadic}.{'_'.join(concept_id)}", True)  # parent concept
            elif concept.count("PositionerName") == 1:
                return (variadic, f"{variadic}.{'_'.join(concept_id)}", True)  # parent concept
            else:
                return (variadic, f"{variadic}.{'_'.join(concept_id)}", False)  # child concept
        else:
            return (concept, concept, True)
    else:
        return (None, None, None)

In [ ]:
def get_protochips_variadic_concept(key, value):
    # return val is a tuple
    # zeroth entry specifies the prefix of the instance concept
    # first entry specifies associated parent concept
    # second entry is True is token resolves an AXON parent concept
    if isinstance(key, str) and key != "":
        concept = key.strip()
        idxs = re.finditer(".\[[0-9]+\].", concept)
        is_variadic = False
        if (sum(1 for _ in idxs) > 0):
            is_variadic = True
            variadic = concept
            concept_id = []
            for idx in re.finditer(".\[[0-9]+\].", concept):
                variadic = variadic.replace(concept[idx.start(0):idx.end(0)], ".[*].")
                concept_id.append(f"{concept[idx.start(0):idx.end(0)]}".replace(".[", "").replace("].", ""))
            print(f"concept_id: {concept_id}")
            if variadic.endswith(".Name"):
                variadic = variadic.replace(".Name", f".{value}")
            return variadic
        else:
            return concept
    return None

In [ ]:
import re
key = "MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[20].HeatingPower"
def specific_to_variadic(token):
    if isinstance(token, str) and key != "":
        concept = token.strip()
        idxs = re.finditer(r".\[[0-9]+\].", concept)
        if (sum(1 for _ in idxs) > 0):
            variadic = concept
            for idx in re.finditer(".\[[0-9]+\].", concept):
                variadic = variadic.replace(concept[idx.start(0):idx.end(0)], ".[*].")
            return variadic
        else:
            return concept
    return None
print(specific_to_variadic(key))

In [ ]:
key = "MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[20].Name"
value = "HeatingPower"
key = "MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[20].Value"
value = 0.077978329305
key = "MicroscopeControlImageMetadata.@xmlns:i"
value = "http://www.w3.org/2001/XMLSchema-instance"
key = "MicroscopeControlImageMetadata.ImagerSettings.ImagePhysicalSize.X"
value = 0.99
key = "MicroscopeControlImageMetadata.ActivePositionerSettings.PositionerSettings.[0].X"
value = -369.84159375

# first phase analyse the collection of Protochips metadata concept instance symbols and reduce to unique concepts
concepts = {}
for concept, value in flattened_xml.items():
    # not every key is allowed to define a concept
    # print(f"{concept}: {value}")
    idxs = re.finditer(".\[[0-9]+\].", concept)
    if (sum(1 for _ in idxs) > 0):  # is_variadic (allowed to define eventually a key!)
        markers = [".Name", ".PositionerName"]
        for marker in markers:
            if concept.endswith(marker):
                concepts[f"{concept[0:len(concept)-len(marker)]}"] = value  # .{value}"] = None
    else:
        # not variadic, i.e. defines a concept
        concepts[concept] = value
#for c, v in concepts.items():
#    print(f"{c}: {v}")
print("1. done")
# second phase, evaluate each concept instance symbol wrt to its prefix coming from the unique concept
for k, v in flattened_xml.items():
    # print(f"-->{k}: {v}")
    grpnms = None
    idxs = re.finditer(".\[[0-9]+\].", k)
    if (sum(1 for _ in idxs) > 0):  # is variadic
        search_argument = k[0:k.rfind("].")+1]
        for parent_grpnm, child_grpnm in concepts.items():
            if parent_grpnm.startswith(search_argument):
                grpnms = (parent_grpnm, child_grpnm)
                break
        if grpnms is not None:
            if len(grpnms) == 2:
                if "PositionerSettings" in k and k.endswith(".PositionerName") is False:
                    print(f"vv: {grpnms[0]}.{grpnms[1]}{k[k.rfind('.') + 1:]}: {v}")
                if k.endswith(".Value"):
                    print(f"vv: {grpnms[0]}.{grpnms[1]}: {v}")
    else:
        print(f"nv: {k}: {v}")
# token = "MicroscopeControlImageMetadata.ActivePositionerSettings.PositionerSettings.[0].PositionerName = Stage"
# token = ""
# idxs = re.finditer(".\[[0-9]+\].", token)
# print(sum(1 for _ in idxs))
# a, b, c = instance_to_concept(token)
# print(f"{a}, {b}, {c}")
# print(get_protochips_variadic_concept(key, value))

In [ ]:
def get_axon_concept_prefix(glossary, what):
    # return val is a tuple
    # zeroth entry specifies the prefix of the instance concept
    # first entry specifies associated parent concept
    # second entry is True is token resolves an AXON parent concept
    for term in glossary:
        if token.count(" = ") == 1:
            concept = term.split("=")[0].strip()
            value = term.split("=")[1].strip()
            if concept.endswith("Name") and what in value:
                return concept[0:len(concept)-len("Name")-1]
    return None

Human understand that the following metadata structure from AXON suggests that:
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].Name = SystemStatus
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].Value = Ready
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].ValueType = String
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].ValueUnits = None
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].DisplayUnits = None
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].Description = None
MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].MeasurementTypeName = None

Is equivalent to
*MicroscopeControlImageMetadata.SystemStatus* being a group  # NeXus concept name
MicroscopeControlImageMetadata.SystemStatus.Value = Ready  # instance data
MicroscopeControlImageMetadata.SystemStatus.ValueType = String  # NeXus value category
MicroscopeControlImageMetadata.SystemStatus.ValueUnits = None   # unit
MicroscopeControlImageMetadata.SystemStatus.DisplayUnits = None  # essentially NOMAD MetaInfo display unit relevant for the GUI
MicroscopeControlImageMetadata.SystemStatus.Description = None  # essentially the NeXus docstring
MicroscopeControlImageMetadata.SystemStatus.MeasurementTypeName = None  # no equivalent in NeXus

In [ ]:
import os
import mmap
for file_name in os.listdir(f"{src}/axon"):
    with open(f"{src}/axon/{file_name}", 'rb', 0) as fp:
        s = mmap.mmap(fp.fileno(), 0, access=mmap.ACCESS_READ)
        magic = s.read(8)
        if magic == b'\x89PNG\r\n\x1a\n':  # https://en.wikipedia.org/wiki/List_of_file_signatures
            print(f"{file_name} is PNG")

In [ ]:
import os
import mmap
import PIL
import datetime
events = []
for file_name in os.listdir(f"{src}/axon"):
    # if we try to open non-image files with PIL directly we get an error so one should hunt first on the magic number
    with open(f"{src}/axon/{file_name}", 'rb', 0) as file:
        s = mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ)
        magic = s.read(8)
        if magic != b'\x89PNG\r\n\x1a\n':
            continue
                
    with Image.open(f"{src}/axon/{file_name}", mode="r") as fp:
        # if f"{file_name}" != "20210426T225128.453Raw382.png":  # "20210426T224913.001Raw257.png":
        #    continue
        if isinstance(fp, PIL.PngImagePlugin.PngImageFile) is True:
            print(f"Parsing {src}/axon/{file_name}...")
            fp.load()  # Needed only for .png EXIF data (see citation above)
            # print(fp.info.keys())

            if "MicroscopeControlImage" in fp.info.keys():
                # print(fp.info["MicroscopeControlImage"])
                xml_content = xmltodict.parse(fp.info["MicroscopeControlImage"])
                flattened_xml = flatten_dict(xml_content)
                # first display all instance metadata in their unmodified AXON-specific representation
                axon_concepts = []
                for k,v in flattened_xml.items():
                    # print('{} = {}'.format(k,v))
                    print(f"{k}: {v}")
                    continue
                    if "AuxiliaryDataValue" in f"{k}":
                        axon_concepts.append(f"{k}.{v}")
                    else:
                        axon_concepts.append(f"{k}")
                    # if f"{k}".endswith(".Name"):
                    #     axon_concepts.append(f"{k[0:len(k)-len('Name')-1]}")
                    # else:
                    #     axon_concepts.append(f"{k}")
                # for concept in axon_concepts:
                #     print(concept)
                # print(axon_concepts)
                reqs = ["MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[0].Name.SystemStatus",
                        "MicroscopeControlImageMetadata.MicroscopeDateTime",
                        "MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[7].Name.HolderTemperature"]
                for req in reqs:
                    if req not in axon_concepts:
                        raise ValueError(f"{src}/axon/{file_name} does not contain concept instance {req} !")
                # all required concepts have instance data so parse
                # print(f"{flattened_xml['MicroscopeControlImageMetadata.MicroscopeDateTime']}")
                if f"{flattened_xml['MicroscopeControlImageMetadata.MicroscopeDateTime']}".count(".") == 1:
                    datetime_obj = datetime.datetime.strptime(f"{flattened_xml['MicroscopeControlImageMetadata.MicroscopeDateTime']}", '%Y-%m-%dT%H:%M:%S.%f%z')
                else:
                    datetime_obj = datetime.datetime.strptime(f"{flattened_xml['MicroscopeControlImageMetadata.MicroscopeDateTime']}", '%Y-%m-%dT%H:%M:%S%z')
                events.append((f"{file_name}", datetime_obj))
                # print(f"converted to datetime: {datetime_obj}")
                print(f"{flattened_xml['MicroscopeControlImageMetadata.AuxiliaryData.AuxiliaryDataCategory.[0].DataValues.AuxiliaryDataValue.[7].Value']}")
                fp.seek(0)
                nparr = np.array(fp)
                print(f"{np.shape(nparr)}, {nparr.dtype}")
                continue

            if True is False:
                # identify all unique axon_concepts
                axon_concepts = []
                for k,v in flattened_xml.items():
                    prefix, parent, is_parent = instance_to_concept('{} = {}'.format(k,v))
                    if is_parent is True:
                        if parent not in axon_concepts:
                            axon_concepts.append(parent)
                        else:
                            raise ValueError(f"Found duplicated axon conceptid {conceptid} !")
                for entry in axon_concepts:
                    print(entry)

                # make members of the same axon concept childs
                for k,v in flattened_xml.items():
                    varname, conceptid, is_parent = instance_to_concept('{} = {}'.format(k,v))
                    if conceptid is not None:                
                        if conceptid in axon_concepts.keys():
                            if is_parent is True:
                                print(f"{axon_concepts[conceptid]}")
                            else:
                                print(f"{axon_concepts[conceptid]}.{varname[varname.rfind('.') + 1:]}")
                        else:
                            raise ValueError(f"Unable to find matching axon parent conceptid {conceptid} !")
                    else:
                        print('{} = {}'.format(k,v))
            # elif fnm.lower().endswith(".png") is True:  # check for mime type instead
            #     print(f"There is no iTXt chunk in {fnm} which has embedded XML within the AXON namespace MicroscopeControlImage!")
            else:
                print(f"There is nothing to harvest here!")
            # print(axon_concepts)

In [ ]:
def sort_ascendingly_by_second_argument_iso8601(tup):
    # convert the list of tuples to a numpy array with data type (object, int)
    arr = np.array(tup, dtype=[('col1', object), ('col2', datetime.datetime)])
    # get the indices that would sort the array based on the second column
    indices = np.argsort(arr['col2'])
    # use the resulting indices to sort the array
    sorted_arr = arr[indices]
    # convert the sorted numpy array back to a list of tuples
    sorted_tup = [(row['col1'], row['col2']) for row in sorted_arr]
    return sorted_tup

new_lst = sort_ascendingly_by_second_argument_iso8601(events)
# events = sorted(events, key=lambda x: x if isinstance(x, datetime.datetime) else datetime.datetime(x.year, x.month, x.day))
time_series_start = new_lst[0][1]
print(f"Time series start: {time_series_start}")
for file_name, iso8601 in new_lst:
    continue
    print(f"{file_name}, {iso8601}, {(iso8601 - time_series_start).total_seconds()} s")
print(f"Time series end: {new_lst[-1][1]}, {(new_lst[-1][1] - time_series_start).total_seconds()} s")
